# Automatic style transfer

The aim of this pipeline is to, given an image, find an image that have a similar content but a different style, and then apply a photorealistic style transfer.

Several requirements are needed to run it:
 - run `python install.py`
 - a gpu (for the segmentation part)

## Image segmentation

### Model

Image segmentation we will use is from the semantic-segmentation-pytorch project available here: https://github.com/CSAILVision/semantic-segmentation-pytorch.git (it is automatically downloaded by `install.py`)

## Segmentation zones content

## Payasage transformation

In [ ]:
from toolbox.path_setup import *
from toolbox.segmentation import *

In [ ]:
name_experiment = "landscape"
_, images_path, _, _ = prepare_experiment(name_experiment)

download_image('https://lemag.nikonclub.fr/wp-content/uploads/2017/07/08.jpg',images_path + '/landscape1.jpg')
download_image('https://www.superprof.fr/blog/file/2017/12/choisir-paysage-photo.jpg',images_path+'/landscape2.jpg')
download_image('https://www.geo.fr/var/geo/storage/images/voyages/vos-voyages-de-reve/namibie-a-travers-le-desert-grandiose-du-namib/route-de-cape-town-a-windhoek/1078719-1-fre-FR/route-de-cape-town-a-windhoek.jpg',images_path + '/landscape3.jpg')
download_image('https://i.pinimg.com/originals/c2/83/2b/c2832b6b72709ad29731b91be002877d.jpg',images_path+'/landscapenight1.jpg')
download_image('https://image.freepik.com/photos-gratuite/route-paysage-nuit-fond-galaxie-montagne-voie-lactee_34263-245.jpg',images_path+'/landscapenight2.jpg')
download_image('http://4everstatic.com/images/dessins/digital-art/paysage-de-nuit,-lac,-reflexion,-lune,-ciel-etoile-197184.jpg',images_path+'/landscapenight3.jpg')               

In [ ]:
img=mpimg.imread(images_path + "/landscape2.jpg")
plt.imshow(img)

In [ ]:
images = ['landscape12.jpg','landscapenight14.jpg']
cli, masks, results = generate_segmentation_cli(name_experiment,images)
! cd semsegpt; python $cli

for image in images:
    res_path = results + "/" + image[:-4] + '.png'
    save_segmentation(res_path, masks)

In [ ]:
plot_segmented_images(results,images)

In [ ]:
topics = get_all_topics(mpimg.imread("examples/paysage/masks/paysage2.png"),k=2)

## Choosing best mate

In [ ]:
import toolbox.google_image_scrapper as scrap
import toolbox.compute_content_losses as ccl
from toolbox.segmentation import *

image = "examples/paysage/images/paysage1.jpg"

In [ ]:
search = get_all_topics(mpimg.imread("examples/paysage/masks/paysage1.png"),k=2)
print(search)

In [ ]:
! ls examples/landscape/masks

In [ ]:
list = ["examples/landscape/masks/landscape1.png", 
        "examples/landscape/masks/landscape2.png", 
        "examples/landscape/masks/landscapenight1.png",
        "examples/landscape/masks/landscapenight2.png"]
calc = ccl.Calculator(device="cuda")
print(calc.distance_matrix(list))

In [ ]:
plt.imshow(calc.distance_matrix(list), cmap='hot', interpolation='nearest')

## Time to do some style transfer !

In [1]:
import torch
import torchvision.models as models
import os

from toolbox.image_preprocessing import image_loader, masks_loader, plt_images
os.chdir("styletransfer")
from neural_style import run_style_transfer
os.chdir("..")

In [ ]:
# pipelines bootstrapping
from PIL import Image
img = Image.open("experiments/paysage/images/paysage1.jpg")
img.save("examples/input/in3.png", "PNG")
img = Image.open("experiments/paysage/images/paysage_nuit1.jpg")
img.save("examples/style/tar3.png", "PNG")
!cp experiments/paysage/masks/paysage1.png examples/segmentation
!mv examples/segmentation/paysage1.png examples/segmentation/in3.png
!cp experiments/paysage/masks/paysage_nuit1.png examples/segmentation
!mv examples/segmentation/paysage_nuit1.png examples/segmentation/tar3.png

### Define experience paramaters

In [13]:
import torch
import torchvision.models as models
import os

from toolbox.image_preprocessing import image_loader, masks_loader, plt_images
os.chdir("styletransfer")
from neural_style import run_style_transfer
os.chdir("..")

name = "5"
num_epochs = 500

quick = False
ghost  = False
quiet = False

content_index = 3
style_index = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
imsize = 256 if torch.cuda.is_available() else 128

style_weight = 1e6
content_weight = 1e2
reg_weight = 1e-11

content_layers = ['4_2']
style_layers = ['1_1','2_1','3_1','4_1']

optimizer = "lbfgs"
learning_rate = 1
momentum = 0.9
weight_decay = 1e-3

scheduler = "step"
lr_step = 10000
lr_decay = 5e-1

query = "main.py -name {} -device {} -num_epochs {} -style_image {} -content_image {} -imsize {} -style_weight {} -content_weight {} -reg_weight {} -content_layers {} -style_layers {} -optimizer {} -lr {} -momentum {} -weight_decay {} -scheduler {} -lr_step {} -lr_decay {}".format(
    name,
    device,
    num_epochs,
    content_index,
    style_index,
    imsize,
    style_weight,
    content_weight,
    reg_weight,
    " ".join(content_layers),
    " ".join(style_layers),
    optimizer,
    learning_rate,
    momentum,
    weight_decay,
    scheduler,
    lr_step,
    lr_decay
  )

if quick:
    query += " -quick"
if ghost:
    query += " -ghost"
if quiet:
    query += " -quiet"

In [ ]:
from main import *

%matplotlib inline

experience = create_experience(query)

parameters = experience["parameters"]
experiment = experience["experiment"]
listener = experience["listener"]
log = experience["log"]
optimizer = experience["optimizer"]
losses = experience["losses"]
model = experience["model"] 
scheduler = experience["scheduler"]

run_experience(experiment, model, parameters, losses, optimizer, scheduler, listener, log)

print()

experiment.input_image.data.clamp_(0, 1)

log.info("Done style transfering over "+str(experiment.epoch)+" epochs!")

if parameters.save_model:
    save_all(experiment,model,parameters,listener)
if not(parameters.ghost):
    save_images(parameters.res_dir+"output.png",experiment.style_image,experiment.input_image,experiment.content_image)

print("All done")

You have entered an experiment name that already exists even though you are not resuming that experiment, do you wish to continue (this will delete the folder: experiments/5/). [y/n] y
----- Experiments Parameters -----
name : 5
quick : False
ghost : False
no_metrics : False
style_image : 3
content_image : 3
input_image : content
imsize : (256, 256)
keep_params : False
base_model : vgg19
device : cuda
content_layers : ['conv4_2']
style_layers : ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1']
num_epochs : 500
style_weight : 1000000.0
content_weight : 100.0
reg_weight : 1e-11
optimizer : lbfgs
lr : 1.0
momentum : 0.9
weight_decay : 0.001
scheduler : step
lr_step : 10000
lr_decay : 0.5
seed : 42
no_log : False
save_model : True
verbose : True
work_dir : 
save_name : 5
res_dir : experiments/5/
save_parameters_path : experiments/5/save/parameters.json
save_model_path : experiments/5/save/model.pt
save_experiment_path : experiments/5/save/experiment.dat
save_listener_path : experiments/5/save/l

In [ ]:
plt_images(experiment.style_image,experiment.input_image,experiment.content_image)

In [ ]:
if not(parameters.no_metrics):
    generate_plots(parameters, listener)